## Extracting JSON data

**Author:** Benjamin Aw  
**Date:** 8 Dec 2021  
**Context:** Creating new datasets in the form of CSV files for pre-training/training purposes  
**Objective:** Compartmentilising the extracted files from JSON into CSV files split up by year-month  

#### A) Setting up

Importing the libraries and setting up the path which contains the raw datasets

In [ ]:
import os
import json
import pandas as pd

path = "../Data/Raw/"

#### B) Writing out the necessary functions

The function below aims to extact out infomation in the json file and compartmentalise it into a dictionary.

In [ ]:
def extract_mcf_data(json):
    
    output = {}
    transfer = ['uuid', 'title', 'description', 'minimumYearsExperience', 'numberOfVacancies']
    # Extracting general information of the job posting
    for key in transfer:
        try:
            output[key] = json[key]
        except:
            # If keys not found, treat file as failure to extract
            return None, None

    # Extract skills, skills are mainly captured in separate JSON objects 
    output['skills'] = ', '.join([entry['skill'] for entry in json['skills']])
    
    # Extract hiring company
    company = ['name', 'description', 'ssicCode', 'employeeCount']
    if json['metadata']['isPostedOnBehalf']:
        company_col = 'hiringCompany'
    else:
        company_col = 'postedCompany'
    for key in company:
        try:
            output['company_' + key] = json[company_col][key]
        except TypeError:
            output['company_' + key] = json[company_col]
        
    # Extract additional infomation such as the date of the post, number of views and applications etc
    metadata = ['originalPostingDate', 'newPostingDate', 'expiryDate', 'totalNumberOfView', 'totalNumberJobApplication']
    for key in metadata:
        output[key] = json['metadata'][key]
    
    # Extract salary, if min and max is not available, return None which is captured in the except statement
    salary = ['maximum', 'minimum']
    for key in salary:
        try:
            output['salary_' + key] = json['salary'][key]
        except TypeError:
            output['salary_' + key] = json['salary']
    
    # Extract additional salary information
    try:
        output['salary_type_id'] = json['salary']['type']['id']
        output['salary_type'] = json['salary']['type']['salaryType']
    except TypeError:
        output['salary_type_id'] = json['salary']
        output['salary_type'] = json['salary']
        
    # Return the actual output, and the date of the post       
    return output, output['originalPostingDate']

The function below runs `extract_mcf_data` and splits up the data based on its year and month that it was posted

In [ ]:
def extract_and_split(path):
    
    output = {}

    for filename in os.listdir(path + "mcf_raw"):    
        
        print(f'Reading in {filename}')
        f = open(path + "/mcf_raw/" + filename)
        entry = json.load(f)
        
        extracted_result, date = extract_mcf_data(entry)

        if extracted_result:
            date_year_mth = date[0:7]
            if date_year_mth in output: 
                output[date_year_mth].append(extracted_result)
            else:
                output[date_year_mth] = [extracted_result]
        else:
            print(f'{filename} has missing key values')
            fi = open(path + "json_to_remove.txt", "a")
            fi.write(f'{filename}\n')
            fi.close()
    
    return output

The function below takes the output from `extract_and_split` and turns them into csv files based on the year and the month of the job posting, which is captured as keys in `output`.

In [ ]:
def write_to_csv(path, output):
    
    for dates in output.keys():
        pd.DataFrame(output[dates]).to_csv(path + "raw_csv/raw_" + dates + ".csv", index = False)

#### C) Putting it all together

Running the code to get the extacted dictionary

In [ ]:
output = extract_and_split(path)

Checking the difference in length between the input and the output. 

This will give us an idea of how much entries are actually dropped. 

In [ ]:
before_len = len(os.listdir(path + "mcf_api_responses"))
bad_len = len(open(path + "json_to_remove.txt", "r").readlines())

print(f'Total number of entries orginally: {before_len}')

print(f'Number of bad entries: {bad_len}')

print(f'Total number of entries after dropping: {before_len - bad_len}')

Writing the outputs into individual csv files 

In [ ]:
write_to_csv(path, output)

#### E) Run only once

Because all the faulty json files are saved into `json_to_remove.txt`, we need to remove them from our current datasets. Trying to remove them concurrently will result in a windows access error.

In [ ]:
f = open(path + "json_to_remove.txt", "r")

for files in f:
    file = files.strip()
    print(f'{file} moved')
    os.rename(f"{path}mcf_raw/{file}", f"{path}json_to_remove/{file}")